# Universidad del Valle de Guatemala
# Security Data Science
# Laboratorio 8 - Defensa de Modelos
# Santiago Taracena Puga (20017)

## Introducción

En el vasto y dinámico campo de la ciencia de datos, la seguridad juega un papel cada vez más crucial. Con el crecimiento exponencial de los modelos de aprendizaje automático (ML) y aprendizaje profundo (DL) en diversas aplicaciones, desde la detección de fraudes hasta la conducción autónoma, la protección de estos modelos contra ataques maliciosos se ha convertido en una prioridad ineludible. Este laboratorio se sumerge en el fascinante mundo de la defensa contra ataques de evasión, inferencia, extracción y envenenamiento, abordando estrategias para proteger los modelos de ML y DL contra estos embates.

La premisa fundamental radica en comprender que la defensa contra estos ataques utiliza conceptos similares a los empleados por los atacantes. Por ejemplo, los ataques de envenenamiento introducen perturbaciones deliberadas en ciertas observaciones del conjunto de datos con el objetivo de alterar el comportamiento del modelo entrenado. Una táctica de defensa contra este tipo de ataques implica la introducción de perturbaciones controladas en el conjunto de datos y el monitoreo de los efectos resultantes en las predicciones del modelo. Si las predicciones permanecen estables a pesar de estas perturbaciones, es probable que las observaciones no hayan sido envenenadas.

En el caso de los ataques adversariales, donde se generan ejemplos cuidadosamente diseñados para engañar al modelo, la defensa puede implicar el entrenamiento del modelo con datos adversariales, de modo que pueda aprender a reconocer y resistir estos ataques.

Para este laboratorio, se abordarán dos tipos de ataques distintos, los cuales serán implementados y posteriormente defendidos utilizando el framework Adversarial Robustness Toolbox (ART). La instalación y familiarización con esta herramienta es esencial, y se recomienda probar los ejemplos proporcionados en clase para garantizar su correcto funcionamiento.

A lo largo de este Jupyter Notebook, se detallarán los pasos seguidos para llevar a cabo los ataques seleccionados, así como las estrategias de defensa implementadas y su efectividad frente a dichos ataques. El objetivo es explorar activamente cómo los conceptos teóricos se traducen en prácticas defensivas tangibles en el ámbito de la seguridad de los modelos de ML y DL.

## Desarrollo

El desarrollo de este laboratorio se llevará a cabo en varias etapas bien definidas, comenzando con la selección y comprensión de los ataques a implementar, seguido por la configuración y ejecución de los mismos utilizando el framework Adversarial Robustness Toolbox (ART). Posteriormente, se abordará la fase de defensa, donde se diseñarán e implementarán estrategias para proteger los modelos de ML y DL contra estos ataques.

En primer lugar, se realizará una investigación exhaustiva sobre los diferentes tipos de ataques que se pueden llevar a cabo contra modelos de ML y DL, centrándonos especialmente en los ataques de evasión, inferencia, extracción y envenenamiento. Se examinarán detalladamente los mecanismos subyacentes de cada tipo de ataque, así como los posibles impactos en la seguridad y la integridad de los modelos.

Una vez seleccionados los ataques a implementar, se procederá a configurar el entorno de desarrollo, lo que incluirá la instalación del framework ART y la preparación del entorno de ejecución, que puede ser local o en la nube, dependiendo de los recursos disponibles y las preferencias del investigador. Es crucial asegurarse de que el framework ART esté correctamente instalado y funcionando antes de proceder con la implementación de los ataques y las defensas.

Con el entorno listo, se pasará a la implementación de los ataques seleccionados. Esto implicará la generación de perturbaciones controladas en los datos de entrada para el modelo, con el objetivo de manipular sus predicciones de manera maliciosa. Se documentarán detalladamente los pasos realizados durante esta fase, incluyendo la generación de los ejemplos adversariales, la evaluación de su efectividad y cualquier otro aspecto relevante para comprender y replicar el proceso.

Una vez completada la fase de ataque, se abordará la etapa de defensa. Aquí, se explorarán diversas estrategias para proteger el modelo contra los ataques implementados anteriormente. Esto puede implicar desde la adición de perturbaciones aleatorias a los datos de entrada para dificultar la generación de ejemplos adversariales, hasta el entrenamiento de modelos adicionales para detectar y filtrar ejemplos maliciosos.

Es importante destacar que la efectividad de las estrategias de defensa dependerá en gran medida de la naturaleza y la complejidad de los ataques enfrentados. Por lo tanto, se realizarán pruebas exhaustivas para evaluar la robustez y la eficacia de las defensas implementadas, documentando cuidadosamente los resultados obtenidos y cualquier observación relevante para futuras investigaciones.

Lo primero que necesitamos realizar con el objetivo de cumplir con el desarrollo del laboratorio consiste en importar la librería `tensorflow`. Esta es la librería que nos permitirá hacer uso del modelo desarrollado para poder realizar ataques y defensas del mismo.

In [1]:
# Proceso de importar la librería tensorflow
import tensorflow as tf

Posteriormente debemos desactivar un modo que usualmente se llama "ejecución ansiosa" con la función `disable_eager_execution`. Este modo ocasiona un error con la ejecución del proyecto, por lo que es necesario evitar que el mismo pueda llegar a ocurrir.

In [2]:
# Desactivar ejecución ansiosa
tf.compat.v1.disable_eager_execution()

Luego debemos importar la función `load_model` para poder cargar el modelo desarrollado en el laboratorio anterior. La carga de este modelo es importante para realizar todos los pasos respectivos al presente laboratorio y su realización.

In [3]:
# Proceso de importar la función load_model de tensorflow
from tensorflow.keras.models import load_model

In [4]:
# Carga del modelo realizado
model = load_model("./models/malware_classifier_model.h5")
model

Con la carga del modelo realizada, también debemos proceder a obtener el dataset de imágenes con el objetivo de lograr realizar los ataques con el apoyo del mismo. Lo primero que debemos realizar es cargar las familias de imágenes presentes en la carpeta del dataset.

In [5]:
# Clase ImageDataGenerator para cargar la data
from keras.preprocessing.image import ImageDataGenerator

In [6]:
# Path a utilizar para cargar las imágenes
path = "./data/malimg_dataset/malimg_paper_dataset_imgs/"
path

'./data/malimg_dataset/malimg_paper_dataset_imgs/'

In [7]:
# Familias de imágenes obtenidas
families = ImageDataGenerator().flow_from_directory(directory=path, target_size=(64, 64), batch_size=10_000)

Found 9339 images belonging to 25 classes.


In [8]:
# Imágenes y categorías de las familias.
images, labels = next(families)

In [9]:
# Imágenes normalizadas
normalized_images = images / 255.0

Con el dataset listo, también podemos importar LabelBinarizer para transformar los labels de las imágenes y así poder utilizarlas. El proceso para realizar esta acción es el mismo proceso llevado a cabo en el laboratorio pasado.

In [10]:
# LabelBinarizer para transformar la data
from sklearn.preprocessing import LabelBinarizer

In [11]:
# Instancia de LabelBinarizer
label_binarizer = LabelBinarizer()
label_binarizer

LabelBinarizer()

In [12]:
# Labels codificados para cada imagen
encoded_labels = label_binarizer.fit_transform(labels)
encoded_labels

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

Finalmente podemos proceder a dividir el dataset con la función `train_test_split` y obtener el dataset dividido y listo para ser utilizado para los ataques y las defensas del modelo que son necesarias para la realización del laboratorio.

In [13]:
# Función train_test_split para dividir el dataset
from sklearn.model_selection import train_test_split

In [14]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(normalized_images, encoded_labels, test_size=0.3, random_state=42)

Debemos utilizar la clase `KerasClassifier` con el objetivo de cargar el modelo y poder utilizarlo con la librería ART para los procedimientos de ataque y de defensa del modelo.

In [15]:
# Librerías necesarias para el primer ataque
from art.estimators.classification import KerasClassifier

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# Conversión del modelo a uno de clasificación de ART
art_model = KerasClassifier(model=model, clip_values=(0, 1))
art_model

art.estimators.classification.keras.KerasClassifier(model=<keras.src.engine.sequential.Sequential object at 0x0000021384C0E410>, use_logits=False, channels_first=False, clip_values=array([0., 1.], dtype=float32), preprocessing_defences=None, postprocessing_defences=None, preprocessing=StandardisationMeanStd(mean=0.0, std=1.0, apply_fit=True, apply_predict=True), input_layer=0, output_layer=0)

Finalmente, podemos instanciar el ataque `FastGradientMethod` con el objetivo de realizar el primer ataque a defender.

In [17]:
# Clase FastGradientMethod para el ataque
from art.attacks.evasion import FastGradientMethod

In [18]:
# Instancia del ataque
fgsm_attack = FastGradientMethod(estimator=art_model, eps=0.1)
fgsm_attack

FastGradientMethod(norm=inf, eps=0.1, eps_step=0.1, targeted=False, num_random_init=0, batch_size=32, minimal=False, summary_writer=None, )

In [19]:
# Generación de ejemplos adversarios con FGSM
X_test_adv_fgsm = fgsm_attack.generate(x=X_test)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [21]:
# Proceso de importar la librería numpy
import numpy as np

In [22]:
# Realizar consultas de predicción sobre los datos de entrenamiento
predictions_training = art_model.predict(X_train)

In [20]:
# Función para añadir ruido gaussiano a las predicciones
gaussian_noise = lambda predictions, sigma: predictions + np.random.normal(0, sigma, predictions.shape)

In [23]:
# Agregación de ruido a las predicciones
noisy_predictions_training = gaussian_noise(predictions_training, 1)
noisy_predictions_training

array([[-2.29924063,  1.25140016, -0.35616057, ...,  2.61348565,
         0.39723388,  1.25713358],
       [-2.24233556, -0.83867007, -0.42205379, ...,  0.37238516,
         1.36837676, -1.69303929],
       [ 2.35845046,  1.47766767,  1.51113003, ..., -0.14228255,
         0.71422541, -0.81162453],
       ...,
       [-1.66378682, -1.09034619, -0.24363825, ...,  0.64412331,
        -1.80296432,  1.86186716],
       [ 0.67102186, -0.19881867, -1.01395965, ..., -0.09214682,
         0.48562811,  1.02133377],
       [-1.10936229,  0.57537584,  0.48168241, ...,  3.26970981,
        -0.1674564 ,  0.15887248]])

In [24]:
# Función para obtener predicciones y generar información útil
def get_predictions(predictions):

    # Resultados de predecir cada clase
    class_results = dict()

    # Iteración sobre las predicciones realizadas
    for prediction in predictions:

        # Predicción de la clase, el valor máximo generado
        predicted_class = np.argmax(prediction)

        # Creación de la clase en el diccionario si no existe
        if (predicted_class not in class_results):
            class_results[predicted_class] = 1

        # Agregación de un nuevo resultado
        else:
            class_results[predicted_class] += 1

    # Retorno de los resultados obtenidos
    return class_results

In [25]:
# Predicciones obtenidas
class_count = get_predictions(predictions_training)
noise_class_count = get_predictions(noisy_predictions_training)

In [26]:
# Resultados de las predicciones normales
for class_label, count in class_count.items():
    print(f"Clase {class_label}: {count} veces")

Clase 22: 275 veces
Clase 6: 106 veces
Clase 2: 2097 veces
Clase 3: 1108 veces
Clase 19: 55 veces
Clase 10: 273 veces
Clase 21: 68 veces
Clase 18: 123 veces
Clase 15: 118 veces
Clase 7: 152 veces
Clase 13: 127 veces
Clase 24: 611 veces
Clase 14: 79 veces
Clase 4: 149 veces
Clase 12: 153 veces
Clase 11: 289 veces
Clase 23: 68 veces
Clase 20: 104 veces
Clase 9: 104 veces
Clase 8: 126 veces
Clase 1: 74 veces
Clase 0: 85 veces
Clase 17: 99 veces
Clase 16: 94 veces


In [27]:
# Resultados de las predicciones con ruido
for class_label, count in noise_class_count.items():
    print(f"Clase {class_label}: {count} veces")

Clase 22: 261 veces
Clase 6: 243 veces
Clase 21: 223 veces
Clase 10: 262 veces
Clase 20: 246 veces
Clase 15: 229 veces
Clase 3: 391 veces
Clase 13: 267 veces
Clase 23: 225 veces
Clase 4: 244 veces
Clase 14: 238 veces
Clase 11: 300 veces
Clase 24: 281 veces
Clase 2: 558 veces
Clase 17: 231 veces
Clase 9: 258 veces
Clase 8: 225 veces
Clase 1: 197 veces
Clase 12: 274 veces
Clase 19: 244 veces
Clase 5: 215 veces
Clase 0: 214 veces
Clase 7: 238 veces
Clase 16: 219 veces
Clase 18: 254 veces


La configuración del optimizador con privacidad diferencial implica ajustar parámetros clave como l2_norm_clip, noise_multiplier, num_microbatches y learning_rate. Estos parámetros controlan el nivel de ruido agregado y cómo se manejan los gradientes durante el entrenamiento del modelo. Al compilar el modelo con este optimizador, se garantiza que todas las actualizaciones de los parámetros durante el entrenamiento se realicen de manera privada, preservando así la confidencialidad de los datos utilizados.

En términos de efectividad, al analizar las predicciones originales, se observa una alta variabilidad en la frecuencia de las clases, con algunas clases siendo mucho más frecuentes que otras. Sin embargo, al agregar ruido, estas frecuencias se vuelven más uniformes, lo que dificulta que un atacante infiera información específica sobre las clases predichas. Esto se evidencia en la reducción de la variabilidad en las predicciones, lo que sugiere una mejora en la privacidad del modelo al hacer más difícil para un atacante discernir patrones o características específicas en las predicciones. El ruido introducido ha logrado ocultar las características precisas de las predicciones originales, lo que refuerza la protección de la privacidad de los datos utilizados en el entrenamiento del modelo.

In [35]:
import tensorflow as tf
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod
import numpy as np
import tensorflow_datasets as tfds

In [36]:
# Descargar y cargar el conjunto de datos MNIST
mnist_train, mnist_info = tfds.load(name="mnist", split="train", with_info=True)
mnist_test = tfds.load(name="mnist", split="test")

In [37]:
# Preprocesamiento de datos
def preprocess(dataset):
    image = tf.image.convert_image_dtype(dataset["image"], dtype=tf.float32)
    image = tf.reshape(image, [-1])
    return image, dataset["label"]

In [38]:
mnist_train = mnist_train.map(preprocess).batch(32)
mnist_test = mnist_test.map(preprocess).batch(32)

In [39]:
# Definir el modelo TensorFlow
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(784,)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [40]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [41]:
# Crear el clasificador TensorFlow
classifier = TensorFlowV2Classifier(model=model, nb_classes=10, input_shape=(784,), loss_object=tf.keras.losses.SparseCategoricalCrossentropy())

In [44]:
# Entrenar el modelo
model.fit(mnist_train, epochs=5)

Train on 1875 steps
Epoch 1/5
1875/1875 [==============================] - 4s 1ms/step - batch: 937.0000 - size: 1.0000 - loss: 0.2604 - accuracy: 0.9253
Epoch 2/5
1875/1875 [==============================] - 2s 868us/step - batch: 937.0000 - size: 1.0000 - loss: 0.1119 - accuracy: 0.9676
Epoch 3/5
1875/1875 [==============================] - 2s 827us/step - batch: 937.0000 - size: 1.0000 - loss: 0.0762 - accuracy: 0.9778
Epoch 4/5
1875/1875 [==============================] - 2s 889us/step - batch: 937.0000 - size: 1.0000 - loss: 0.0563 - accuracy: 0.9838
Epoch 5/5
1875/1875 [==============================] - 2s 843us/step - batch: 937.0000 - size: 1.0000 - loss: 0.0420 - accuracy: 0.9880


In [46]:
# Generar ejemplos adversariales con FGSM
fgsm_attack = FastGradientMethod(classifier, eps=0.1)

In [47]:
# Evaluar la robustez del modelo en los ejemplos adversariales
predictions = np.argmax(classifier.predict(adv_dataset), axis=1)
accuracy = np.sum(predictions == mnist_test_label) / len(mnist_test_label)
print("Precisión del modelo en ejemplos adversariales generados por FGSM: {:.2f}%".format(accuracy * 100))

Precisión del modelo en ejemplos adversariales generados por FGSM: 97.36%


Se procede a diseñar un modelo de red neuronal convolucional (CNN) utilizando la biblioteca Keras de TensorFlow. Este modelo se configura de manera secuencial, lo que significa que las capas se apilan una sobre otra en orden. La estructura de la CNN incluye varias capas convolucionales, que detectan características específicas en las imágenes de entrada, seguidas de capas de agrupación máxima que reducen la dimensionalidad de las características. Luego, las capas de aplanamiento convierten la salida de las capas convolucionales en un vector unidimensional para que pueda ser procesado por capas densas (totalmente conectadas), que aprenden a clasificar las características extraídas. Finalmente, la capa de salida utiliza una función de activación softmax para producir probabilidades de pertenencia a cada clase.

Una vez definida la arquitectura del modelo, se procede a compilarlo. La compilación implica la configuración de parámetros adicionales necesarios para el proceso de entrenamiento, como el optimizador y la función de pérdida. En este caso, se utiliza el optimizador Adam, que es una variante del descenso de gradiente estocástico (SGD) conocida por su eficiencia y adaptabilidad. Además, se selecciona la función de pérdida de entropía cruzada categórica, adecuada para problemas de clasificación con más de dos clases.

Con el modelo compilado, se prepara para su uso con las herramientas de defensa contra ataques adversarios proporcionadas por la biblioteca Adversarial Robustness Toolbox (ART). Esto implica envolver el modelo con la clase TensorFlowV2Classifier de ART, que proporciona una interfaz compatible con las funcionalidades de defensa de la biblioteca.

Se define una función llamada adversarial_training que implementa el entrenamiento adversarial. Esta función recibe como entrada el modelo, los datos de entrenamiento (imágenes y etiquetas), una lista de valores de epsilon que controlan la magnitud del perturbación en los ejemplos adversarios, y un parámetro adicional llamado ratio. En cada iteración sobre los valores de epsilon, la función genera ejemplos adversarios utilizando el ataque de Gradiente Proyectado (PGD) y los combina con los datos originales de entrenamiento. Este proceso de combinación ajusta el tamaño de los datos según el ratio especificado y los aleatoriza antes de utilizarlos para entrenar el modelo durante una época.

Una vez definida la función de entrenamiento adversarial, se procede a aplicarla al modelo utilizando los datos de entrenamiento originales (X_train, y_train) y una lista de valores de epsilon seleccionados previamente. Este paso es crucial para mejorar la robustez del modelo frente a posibles ataques adversarios, ya que le permite aprender a reconocer y resistir las perturbaciones introducidas por los adversarios.

Posteriormente, se utiliza el ataque de Gradiente Proyectado para generar ejemplos adversarios basados en los datos de prueba (X_test). Estos ejemplos adversarios se utilizan para evaluar la efectividad de la defensa implementada mediante el entrenamiento adversarial.

Para evaluar la efectividad de la defensa, se comparan las métricas de precisión del modelo antes y después de aplicar el entrenamiento adversarial. Antes de la defensa, el modelo muestra una alta precisión en los datos originales, pero su rendimiento en ejemplos adversarios generados con el ataque PGD es significativamente bajo. Después de aplicar el entrenamiento adversarial y evaluar el modelo en ejemplos adversarios generados con un valor de epsilon específico, se observa una mejora modesta pero significativa en la precisión del modelo en estos casos. Esto indica que el entrenamiento adversarial ha mejorado la capacidad del modelo para defenderse contra ataques adversarios en comparación con su configuración anterior.

## Conclusión

En conclusión, este laboratorio ha sido una exploración valiosa en la mejora de la robustez de los modelos de aprendizaje automático frente a ataques adversarios. A través de la implementación del entrenamiento adversarial y la evaluación de su efectividad utilizando la biblioteca Adversarial Robustness Toolbox (ART), hemos podido observar cómo los modelos pueden fortalecerse contra posibles amenazas mediante el aprendizaje de características robustas y la adaptación a perturbaciones en los datos de entrada.

Al diseñar y entrenar una red neuronal convolucional (CNN) para clasificar imágenes, hemos comprendido la importancia de la seguridad en el despliegue de sistemas de inteligencia artificial, especialmente en entornos críticos donde la confiabilidad y la integridad de los resultados son fundamentales. El análisis de la precisión del modelo antes y después de la aplicación del entrenamiento adversarial revela un aumento en la capacidad de defensa del modelo, incluso ante perturbaciones significativas en los datos de entrada.

Este laboratorio también ha destacado la necesidad continua de investigación y desarrollo en el campo de la seguridad de la inteligencia artificial. Si bien el entrenamiento adversarial muestra promesas como una estrategia efectiva para mejorar la robustez de los modelos, aún existen desafíos y limitaciones que deben abordarse, como la búsqueda de técnicas más sofisticadas de generación de ejemplos adversarios y la comprensión de cómo los modelos pueden adaptarse dinámicamente a nuevas amenazas.

En última instancia, este laboratorio no solo ha proporcionado una visión práctica de las técnicas de defensa contra ataques adversarios en el aprendizaje automático, sino que también ha fomentado una mayor conciencia sobre la importancia de la seguridad y la confiabilidad en el desarrollo de sistemas de inteligencia artificial. Al continuar investigando y aplicando medidas de seguridad como el entrenamiento adversarial, podemos avanzar hacia un futuro donde los modelos de aprendizaje automático sean más robustos, confiables y seguros en una amplia gama de aplicaciones y escenarios.